### Join strong host factors together

In [66]:
import pandas as pd

data = pd.ExcelFile("/home/samuele/GhostFreePro/data_preprocessing_pipeline/artivir_data/ARTIvir_CoV_minimal_combined_dset.xlsx")

In [67]:
data.sheet_names

['PPIs',
 'EFF',
 'Tx',
 'FP',
 'Ux',
 'PPx',
 'NHBE_FPs',
 'Internal_SARSCoV2',
 'HostFactors_Baggen_A549s',
 'HostFactors_Baggen_2_studies']

In [68]:
dfs =  {sheet_name: data.parse(sheet_name) for sheet_name in data.sheet_names}

/home/samuele/mambaforge/envs/pytorch_geometric/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/home/samuele/mambaforge/envs/pytorch_geometric/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/samuele/mambaforge/envs/pytorch_geometric/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/home/samuele/mambaforge/envs/pytorch_geometric/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/samuele/mambaforge/envs/pytorch_geometric/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/home/samuele/m

In [70]:
dfs.keys()

dict_keys(['PPIs', 'EFF', 'Tx', 'FP', 'Ux', 'PPx', 'NHBE_FPs', 'Internal_SARSCoV2', 'HostFactors_Baggen_A549s', 'HostFactors_Baggen_2_studies'])

In [71]:
df_host_factors_baggen = dfs["HostFactors_Baggen_A549s"]

In [72]:
df_host_factors_baggen.head()

,Reference,Assay type,Assay cell line,Method,Bait,Gene name,Uniprot accession,Functionally validated by authors,Validated in cell line
0,Gordon et al.,Protein interactome,HEK293T,AP-MS,orf8,ADAM9,Q13443,Yes,A549-ACE2
1,Gordon et al.,Protein interactome,HEK293T,AP-MS,nsp4,ALG11,Q2TAA5,Yes,A549-ACE2
2,Gordon et al.,Protein interactome,HEK293T,AP-MS,nsp15,ARF6,P62330,Yes,A549-ACE2
3,Gordon et al.,Protein interactome,HEK293T,AP-MS,nsp6,ATP6AP1,Q15904,Yes,A549-ACE2/Caco-2
4,Gordon et al.,Protein interactome,HEK293T,AP-MS,orf9c,BCS1L,Q9Y276,Yes,A549-ACE2


In [76]:
import numpy as np
np.unique(df_host_factors_baggen["Assay type"].values)

array(['Functional validation', 'Genome-wide functional screen ',
       'Protein interactome', 'RNA interactome',
       'Targeted functional screen'], dtype=object)

In [83]:
strong_host_factors = np.unique(df_host_factors_baggen[df_host_factors_baggen["Assay type"] != "RNA interactome"][df_host_factors_baggen["Assay type"] != "Protein interactome"]["Gene name"].values)

/tmp/ipykernel_240670/407891942.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  strong_host_factors = np.unique(df_host_factors_baggen[df_host_factors_baggen["Assay type"] != "RNA interactome"][df_host_factors_baggen["Assay type"] != "Protein interactome"]["Gene name"].values)


In [84]:
len(strong_host_factors)

61

In [85]:
with open("/home/samuele/GhostFreePro/data_preprocessing_pipeline/artivir_data/strong_host_factors.txt", "r") as f:
    host_factors_baggen_2_studies = f.read().splitlines()

In [86]:
host_factors_baggen_2_studies = np.unique(host_factors_baggen_2_studies)

In [87]:
len(host_factors_baggen_2_studies)

64

In [152]:
host_factors = np.unique(np.concatenate((strong_host_factors, host_factors_baggen_2_studies))).astype('str')

In [153]:
len(host_factors)

89

In [91]:
np.save("/home/samuele/GhostFreePro/data_preprocessing_pipeline/artivir_data/strong_host_factors.npy", host_factors)    

In [92]:
test = np.load("/home/samuele/GhostFreePro/data_preprocessing_pipeline/artivir_data/strong_host_factors.npy")

In [93]:
len(test)

89

### Select most frequent host factors and functionally validated host factors in publications

In [154]:
host_factors_xlsx = pd.ExcelFile("/home/samuele/GhostFreePro/data_preprocessing_pipeline/artivir_data/host_factors_from_publications.xlsx")

host_factors_dfs = {sheet_name: host_factors_xlsx.parse(sheet_name) for sheet_name in host_factors_xlsx.sheet_names}

In [155]:
host_factors_df = host_factors_dfs["host_factors"]

In [157]:
host_factors_df.head()

,Reference,Assay type,Assay cell line,Method,Bait,Gene name,Uniprot accession,Functionally validated by authors
0,Gordon et al.,Protein interactome,HEK293T,AP-MS,M,AAR2,Q9Y312,NaN
1,Li et al.,Protein interactome,HEK293T,AP-MS,nsp10,AARS1,P49588,NaN
2,Li et al.,Protein interactome,HEK293T,AP-MS,nsp10,AASDHPPT,Q9NRN7,NaN
3,Gordon et al.,Protein interactome,HEK293T,AP-MS,M,AASS,Q9UDR5,NaN
4,Gordon et al.,Protein interactome,HEK293T,AP-MS,nsp8,AATF,Q9NY61,NaN


In [186]:
# list of collected studies
np.unique(host_factors_df["Reference"].values)

array(['Baggen et al.', 'Biering et al.', 'Clausen et al.', 'Daly et al.',
       'Daniloski et al.', 'Davies et al.', 'Flynn et al.',
       'Flynn et al. ', 'Gordon et al.', 'Grodzki et al.',
       'Hoffman et al. ', 'Kamel et al.', 'Kratzel et al.',
       'Labeau et al.', 'Laurent et al.', 'Lee et al.', 'Li et al.',
       'Rebendenne et al.', 'Samavarch-Tehrani et al. ', 'Schmidt et al.',
       'Schneider et al.', 'St Germain et al.', 'Stukalov et al.',
       'Sun et al.', 'Wang Ke et al. ', 'Wang Shuai et al.',
       'Wang et al.', 'Wei Congwen et al.', 'Wei et al.', 'White et al.',
       'Zhu Shiyou et al. ', 'Zhu et al.', 'Zhu et al. '], dtype=object)

In [158]:
host_factors_from_publications = np.unique(host_factors_df["Gene name"].values)

In [159]:
len(host_factors_from_publications)

5075

In [160]:
np.unique(host_factors_df["Assay type"].values)

array(['Functional validation', 'Genome-wide functional screen ',
       'Protein interactome', 'RNA interactome',
       'Targeted functional screen'], dtype=object)

In [183]:
# filter for functionally validated host factors
functionally_validated_host_factors_df = host_factors_df[host_factors_df["Functionally validated by authors"] ==  "Yes"]
functionally_validated_host_factors_df.head()

,Reference,Assay type,Assay cell line,Method,Bait,Gene name,Uniprot accession,Functionally validated by authors
34,Gordon et al.,Protein interactome,HEK293T,AP-MS,orf8,ADAM9,Q13443,Yes
65,Gordon et al.,Protein interactome,HEK293T,AP-MS,nsp4,ALG11,Q2TAA5,Yes
91,Gordon et al.,Protein interactome,HEK293T,AP-MS,nsp15,ARF6,P62330,Yes
139,Gordon et al.,Protein interactome,HEK293T,AP-MS,nsp6,ATP6AP1,Q15904,Yes
166,Gordon et al.,Protein interactome,HEK293T,AP-MS,orf9c,BCS1L,Q9Y276,Yes


In [184]:
len(np.unique(functionally_validated_host_factors_df["Gene name"].values))

196

In [236]:
strong_functionally_validated_host_factors = np.unique(np.concatenate((host_factors, np.unique(functionally_validated_host_factors_df["Gene name"].values)))).astype('str')

In [240]:
len(strong_functionally_validated_host_factors)

198

In [241]:
np.save("/home/samuele/GhostFreePro/data_preprocessing_pipeline/artivir_data/strong_functionally_validated_host_factors.npy", strong_functionally_validated_host_factors)

In [161]:
# filter for assay type, avoid Protein Interactome and RNA interactome biases
len(np.unique(host_factors_df[host_factors_df["Assay type"] !=  "Protein interactome"][host_factors_df["Assay type"] !=  "RNA interactome"]["Gene name"].values))

/tmp/ipykernel_240670/3237915194.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(np.unique(host_factors_df[host_factors_df["Assay type"] !=  "Protein interactome"][host_factors_df["Assay type"] !=  "RNA interactome"]["Gene name"].values))


753

In [162]:
filtered_host_factors_df = host_factors_df[host_factors_df["Assay type"] !=  "Protein interactome"][host_factors_df["Assay type"] !=  "RNA interactome"]
filtered_host_factors_df.head()

/tmp/ipykernel_240670/2204499336.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_host_factors_df = host_factors_df[host_factors_df["Assay type"] !=  "Protein interactome"][host_factors_df["Assay type"] !=  "RNA interactome"]


,Reference,Assay type,Assay cell line,Method,Bait,Gene name,Uniprot accession,Functionally validated by authors
19133,Biering et al.,Genome-wide functional screen,Calu-3,CRISPR KO,NaN,AADACL4,Q5VUY2,NaN
19134,Biering et al.,Genome-wide functional screen,Calu-3,CRISPR KO,NaN,AAGAB,Q6PD74,NaN
19135,Biering et al.,Genome-wide functional screen,Calu-3,CRISPR KO,NaN,ACE2,Q9BYF1,Yes
19136,Rebendenne et al.,Genome-wide functional screen,Vero-E6,CRISPR KO,NaN,ACE2,Q9BYF1,Yes
19137,Rebendenne et al.,Genome-wide functional screen,Caco-2,CRISPR KO,NaN,ACE2,Q9BYF1,Yes


In [163]:
len(filtered_host_factors_df)

899

In [165]:
# remaining publications after filtering
len(np.unique(filtered_host_factors_df["Reference"].values))

22

In [166]:
unique_genes, pub_counts = np.unique(filtered_host_factors_df["Gene name"].values, return_counts = True)

In [167]:
len(unique_genes)

753

In [168]:
# select filtered host factors shared by at least 2 publications
host_factors_from_pubs = []
for gene, pub_number in zip(unique_genes, pub_counts):
    if pub_number >= 2:
        host_factors_from_pubs.append(gene)

print(len(host_factors_from_pubs))        

101


In [235]:
strong_host_factors_list_from_publications = np.unique(np.concatenate((host_factors, host_factors_from_pubs))).astype('str')
len(strong_host_factors_list_from_publications)

131

In [171]:
len(host_factors)

89

In [172]:
np.save("/home/samuele/GhostFreePro/data_preprocessing_pipeline/artivir_data/strong_host_factors_list_from_publications.npy", strong_host_factors_list_from_publications)    

In [173]:
# select filtered host factors shared by at least 1 publications
weak_host_factors_from_pubs = []
for gene, pub_number in zip(unique_genes, pub_counts):
    if pub_number >= 1:
        weak_host_factors_from_pubs.append(gene)

print(len(weak_host_factors_from_pubs)) 

753


In [174]:
weak_host_factors_list_from_publications = np.unique(np.concatenate((host_factors, weak_host_factors_from_pubs))).astype('str')
len(weak_host_factors_list_from_publications)

755

In [175]:
np.save("/home/samuele/GhostFreePro/data_preprocessing_pipeline/artivir_data/weak_host_factors_list_from_publications.npy", weak_host_factors_list_from_publications)

In [58]:
# I recreate the set of positives starting from Stukalov and then checking the repetitions in other indipendent publications
#stukalov_genes = host_factors_df[host_factors_df["Reference"] == 'Stukalov et al.']["Gene name"].values
#unique_genes, pub_counts = np.unique(host_factors_df["Gene name"].values, return_counts = True)

### Multiscale  interactome Network

In [244]:
df_multiscale_interactome = pd.read_csv("/home/samuele/GhostFreePro/data_preprocessing_pipeline/artivir_data/multiscale_interactome.txt", sep="\t")

In [246]:
df_multiscale_interactome.head()

,node_1,node_2,node_1_type,node_2_type,node_1_name,node_2_name
0,6416,2318,protein,protein,MAP2K4,FLNC
1,6416,9043,protein,protein,MAP2K4,SPAG9
2,6416,5599,protein,protein,MAP2K4,MAPK8
3,6416,5871,protein,protein,MAP2K4,MAP4K2
4,6416,1326,protein,protein,MAP2K4,MAP3K8


In [248]:
# let's recreate the string starting database with gene names with the same used column names
df_example_string = pd.read_csv("/home/samuele/GhostFreePro/data_preprocessing_pipeline/artivir_data/df_string_ppi_with_gene_names.zip")
df_example_string.head()

,protein1,protein2,experimental,database,textmining,combined_score,gene_name_1,gene_name_2
0,9606.ENSP00000000233,9606.ENSP00000264718,157,0,0,156,ARF5,GPN1
1,9606.ENSP00000000233,9606.ENSP00000346046,178,0,0,177,ARF5,RPS17
2,9606.ENSP00000000233,9606.ENSP00000347134,143,0,63,162,ARF5,NET1
3,9606.ENSP00000000233,9606.ENSP00000317469,345,0,92,379,ARF5,BBS1
4,9606.ENSP00000000233,9606.ENSP00000302393,288,0,0,287,ARF5,LDHAL6B


In [249]:
df_multiscale_interactome["node_2_name"].values

array(['FLNC', 'SPAG9', 'MAPK8', ..., 'CHRNB2', 'CPN2', nan], dtype=object)

In [250]:
# percentage nan values links
sum(df_example_string["gene_name_2"].values.astype(str) == 'nan')/len(df_example_string["gene_name_2"].values.astype(str))

0.026825432999088423

In [251]:
# percentage nan values links
sum(df_multiscale_interactome["node_2_name"].values.astype(str) == 'nan')/len(df_multiscale_interactome["node_2_name"].values.astype(str))

0.004852615665615826

In [254]:
df_multiscale_interactome = df_multiscale_interactome.rename({'node_1_name': 'gene_name_1', 'node_2_name': 'gene_name_2'}, axis=1)
df_multiscale_interactome.head()

,node_1,node_2,node_1_type,node_2_type,gene_name_1,gene_name_2
0,6416,2318,protein,protein,MAP2K4,FLNC
1,6416,9043,protein,protein,MAP2K4,SPAG9
2,6416,5599,protein,protein,MAP2K4,MAPK8
3,6416,5871,protein,protein,MAP2K4,MAP4K2
4,6416,1326,protein,protein,MAP2K4,MAP3K8


In [255]:
# add experimental columns of all ones
df_multiscale_interactome["experimental"] = np.ones(len(df_multiscale_interactome))
df_multiscale_interactome.head()

,node_1,node_2,node_1_type,node_2_type,gene_name_1,gene_name_2,experimental
0,6416,2318,protein,protein,MAP2K4,FLNC,1.0
1,6416,9043,protein,protein,MAP2K4,SPAG9,1.0
2,6416,5599,protein,protein,MAP2K4,MAPK8,1.0
3,6416,5871,protein,protein,MAP2K4,MAP4K2,1.0
4,6416,1326,protein,protein,MAP2K4,MAP3K8,1.0


In [256]:
df_multiscale_interactome.to_csv("/home/samuele/GhostFreePro/data_preprocessing_pipeline/artivir_data/df_multiscale_interactome.zip", index=False, compression = dict(method='zip',archive_name='df_multiscale_interactome.csv'))